In [1]:
using CSV, DataFrames, Distances, Formatting

In [2]:
cities = CSV.read("cities_p.csv");

In [3]:
mutable struct Score
    value :: Float64
    dist_path :: Vector{Float64}
    scaled_dist_path ::  Vector{Float64}
end

In [4]:
mutable struct Cities
    xy :: Array{Float64,2}
    nprimes :: Vector{Float64}
end

In [5]:
function get_score(cities, subm_path)
    all_ids = cities[:CityId]
    all_x = cities[:X]
    all_y = cities[:Y]

    incs = 0
    score = 0.0
    pimp = 0.0
    for i in 1:length(subm_path)-1
        c_idx = subm_path[i]+1
        n_idx = subm_path[i+1]+1
        p1 = [all_x[c_idx],all_y[c_idx]]
        p2 = [all_x[n_idx],all_y[n_idx]]
        stepSize = euclidean(p1,p2)
        if i % 10 == 0 && !cities[:primes][subm_path[i]+1]
            pimp += 0.1*stepSize
            stepSize *= 1.1
            incs += 1
        end
        # print(stepSize)
        score += stepSize
    end
    return score, incs, pimp
end

get_score (generic function with 1 method)

In [6]:
function calc_score(cities, list_path, tenth)
    @views xy_cities = cities.xy
    len_path     = length(list_path)
    # Calc Distance
    @views xy_path   = xy_cities[list_path,:]
    @views @inbounds dist_path = sqrt.(sum((xy_path[1:end-1,:] .- xy_path[2:end,:]).^2; dims=2))[:,1]
    
    # List of Primes 0 to (len_path-1)
    # Flag array, is path's from-city number non-prime?
    @views is_path_from_non_prime   = cities.nprimes[list_path][1:end-1]   
    # If both flags are true, *1.1, else * 1.0
    extra = sum(dist_path .* 0.1 .* is_path_from_non_prime .* tenth)
    return sum(dist_path)+extra, extra
end

calc_score (generic function with 1 method)

In [18]:
subm_df = CSV.read("tsp_improved_diff_mip_new.csv"); 
subm_path = subm_df[:Path];
tenth = [(i % 10) == 0 for i in 1:length(subm_path)-1]
cities_csv = CSV.read("cities_p.csv");
subm_path = collect(skipmissing(subm_df[:Path]))
subm_path .+= 1
xy_cities   = zeros(size(cities_csv)[1],2)
xy_cities[:,1] = cities_csv[:X]
xy_cities[:,2] = cities_csv[:Y]
cities = Cities(xy_cities, cities_csv[:nprimes])
println(length(subm_path))
score, extra = calc_score(cities, subm_path, tenth)
println("Extra: ", format( extra, commas=true, precision=2))
formatted_score = format( score, commas=true, precision=2)
println("Score: ", formatted_score)
println("Distance to 1: ", score-1515400.47)
println("Distance to Gold: ", score-1516229)
println("Distance to current: ", score-1516438.83)

197770
Extra: 12,626.65
Score: 1,516,435.52
Distance to 1: 1035.0499248430133
Distance to Gold: 206.51992484298535
Distance to current: -3.310075157089159


In [8]:
1.5334926188288075e6

1.5334926188288075e6